In [1]:
import sys
import pandas as pd
import numpy as np
import MySQLdb
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels
from statsmodels.formula.api import ols
from sklearn.preprocessing import PolynomialFeatures

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm


In [3]:
conn = None # Database Connection

In [4]:
conn = MySQLdb.connect('localhost', 'root', 'Newdelhi*1', 'TalendToMysql', 3306)

print('Connected')
    
# Get a Cursor from the Connection, for traversing the records in result-set
    
# Cursor from the connection wihtout Dictionary details i.e. Tuple of Column Heading and Column Value
# cursor = conn.cursor()
    
# Cursor from the connection with Dictionary Details i.e. Tuple of Column Heading and Column Value
# Example: 'category': 'coffee'
cursor = conn.cursor(MySQLdb.cursors.DictCursor)
    
# Execute MySQL query to fetch the current Database Version via execute()
cursor.execute('SELECT Version()')
    
# Fetch one (current) row into a tuple
version = cursor.fetchone()
    
# Print the Database Version Fetched from above Execute Query and display on the console
print('Database Version: %s' % version) # one-item tuple
    
    #########################################################
    
# Fetch record from the table created above with the help of Select * Query
cursor.execute('Select ItemOutletFact.Item_Identifier,ItemOutletFact.Item_Visibility,ItemOutletFact.Item_Outlet_Sales,ItemOutletFact.Item_MRP,ItemDimension.Item_Weight FROM ItemOutletFact LEFT JOIN ItemDimension ON ItemOutletFact.Item_Identifier = ItemDimension.Item_Identifier LEFT JOIN OutletDimension OD ON ItemOutletFact.Outlet_Identifier = OD.Outlet_Identifier')
MultiLinearData = pd.DataFrame(cursor.fetchall())


#3rd Question
cursor.execute('Select ItemOutletFact.Item_Identifier,ItemOutletFact.Item_Visibility,ItemOutletFact.Item_Outlet_Sales,ItemOutletFact.Item_MRP,ItemDimension.Item_Weight,ItemDimension.Item_Type,ItemDimension.Item_Fat_Content FROM ItemOutletFact LEFT JOIN ItemDimension ON ItemOutletFact.Item_Identifier = ItemDimension.Item_Identifier LEFT JOIN OutletDimension OD ON ItemOutletFact.Outlet_Identifier = OD.Outlet_Identifier')
ClassificationData = pd.DataFrame(cursor.fetchall())
#for row in rows:
 #   print(row)

# If connection still exist


Connected
Database Version: {'Version()': '8.0.18'}


In [ ]:
pd.set_option('max_columns', 100000)
pd.set_option('max_colwidth', -1)
pd.set_option('max_rows', 100000)

In [5]:
Y = MultiLinearData[['Item_Outlet_Sales']]

In [6]:
X = MultiLinearData[['Item_MRP','Item_Visibility','Item_Weight']]

In [7]:
X['Item_Weight'] = pd.to_numeric(X['Item_Weight'], errors='coerce')

/Users/garg/anaconda3/envs/DataManagementEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
X['Item_MRP'] = pd.to_numeric(X['Item_MRP'], errors='coerce')

/Users/garg/anaconda3/envs/DataManagementEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
X['Item_Visibility']=pd.to_numeric(X['Item_Visibility'], errors='coerce')

/Users/garg/anaconda3/envs/DataManagementEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
X = X.fillna(-1)

In [11]:
Y['Item_Outlet_Sales'] = pd.to_numeric(Y['Item_Outlet_Sales'], errors='coerce')

/Users/garg/anaconda3/envs/DataManagementEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
X_Train,X_Test,Y_Train,Y_Test = train_test_split(X,Y,test_size=0.2,random_state=1)

In [13]:
reg = LinearRegression()
reg.fit(X_Train, Y_Train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
Y_pred = reg.predict(X_Test)

In [15]:
print("Mean squared error: %.2f" % mean_squared_error(Y_Test, Y_pred))


Mean squared error: 2064494.88


In [16]:
print('R²: %.2f' % r2_score(Y_Test, Y_pred))

R²: 0.31


In [ ]:
plt.scatter(MultiLinearData['Item_Visibility'], MultiLinearData['Item_Outlet_Sales'], color='red')
plt.title(' Item_Outlet_Sales Vs Item_Visibility', fontsize=14)
plt.xlabel('Item_Visibility', fontsize=14)
plt.ylabel('Item_Outlet_Sales', fontsize=14)
#plt.grid(True)
plt.show()

In [ ]:
plt.scatter(MultiLinearData['Item_MRP'], MultiLinearData['Item_Outlet_Sales'], color='red')
plt.title('Item_Outlet_Sales vs Item_MRP', fontsize=14)
plt.xlabel('Item_MRP', fontsize=14)
plt.ylabel('Item_Outlet_Sales', fontsize=14)
#plt.grid(True)
plt.show()

In [ ]:
plt.scatter(MultiLinearData['Item_Weight'], MultiLinearData['Item_Outlet_Sales'], color='red')
plt.title('Item_Outlet_Sales Vs Item_Weight', fontsize=14)
plt.xlabel('Item_Weight', fontsize=14)
plt.ylabel('Item_Outlet_Sales', fontsize=14)
#plt.grid(True)
plt.show()

#Classification Code

In [17]:
ClassificationData.head(5)

,Item_Identifier,Item_Visibility,Item_Outlet_Sales,Item_MRP,Item_Weight,Item_Type,Item_Fat_Content
0,DRA12,0.07,283.63,143.02,11.6,Soft Drinks,Low Fat
1,DRA24,0.07,327.57,163.89,19.35,Soft Drinks,Regular
2,DRA59,0.21,185.09,183.99,8.27,Soft Drinks,Regular
3,DRB13,0.01,948.77,189.05,6.115,Soft Drinks,Regular
4,DRB25,0.12,214.39,107.09,12.3,Soft Drinks,Low Fat


In [18]:
XClassification = ClassificationData[['Item_Visibility','Item_Outlet_Sales','Item_MRP','Item_Weight']]

In [19]:
YClassification = ClassificationData[['Item_Fat_Content']]

In [20]:
XClassification['Item_Weight'] = pd.to_numeric(XClassification['Item_Weight'], errors='coerce')

/Users/garg/anaconda3/envs/DataManagementEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
XClassification['Item_Weight'] = XClassification['Item_Weight'].fillna(-1)

/Users/garg/anaconda3/envs/DataManagementEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
#XClassification = XClassification.fillna(-1)

In [23]:
XClassification = XClassification.astype(float)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(XClassification,YClassification, test_size=0.20)

#Logistics Regression

In [25]:
logisticRegr = LogisticRegression()

In [26]:
logisticRegr.fit(X_train, y_train)

/Users/garg/anaconda3/envs/DataManagementEnv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/garg/anaconda3/envs/DataManagementEnv/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
model = logisticRegr.predict(X_test)

In [28]:
score = logisticRegr.score(X_test, y_test)
print(score)

0.6645161290322581


In [29]:
print(classification_report(y_test,model))

              precision    recall  f1-score   support

     Low Fat       0.66      1.00      0.80      1133
     Regular       0.00      0.00      0.00       572

    accuracy                           0.66      1705
   macro avg       0.33      0.50      0.40      1705
weighted avg       0.44      0.66      0.53      1705



/Users/garg/anaconda3/envs/DataManagementEnv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [30]:
del X_train, X_test, y_train, y_test

#KNN

In [31]:
X_train, X_test, y_train, y_test = train_test_split(XClassification,YClassification, test_size=0.20)

In [32]:
classifier = KNeighborsClassifier(n_neighbors=5)

In [33]:
classifier.fit(X_train, y_train)

/Users/garg/anaconda3/envs/DataManagementEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [34]:
y_pred = classifier.predict(X_test)

In [35]:
print(confusion_matrix(y_test, y_pred))

[[847 250]
 [434 174]]


In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     Low Fat       0.66      0.77      0.71      1097
     Regular       0.41      0.29      0.34       608

    accuracy                           0.60      1705
   macro avg       0.54      0.53      0.52      1705
weighted avg       0.57      0.60      0.58      1705



In [37]:
classifier.score(X_test,y_test)

0.5988269794721408

In [38]:
del X_train, X_test, y_train, y_test

#SVM

In [39]:
X_train, X_test, y_train, y_test = train_test_split(XClassification,YClassification, test_size=0.20)

In [40]:
clf = svm.SVC(kernel='linear')

In [41]:
clf.fit(X_train, y_train)

/Users/garg/anaconda3/envs/DataManagementEnv/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [42]:
y_pred = clf.predict(X_test)

In [43]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6269794721407624


In [44]:
if conn:
    # Close the database connection
    conn.close()
    print('Closed..')

Closed..
